In [ ]:
# prompt: import pandas as pd

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
!pip install random-forest-mc
!pip install  scikit-learn==0.23.2

  Using cached scikit-learn-0.23.2.tar.gz (7.2 MB)
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [ ]:
train = pd.read_csv('/content/Titanic-Dataset.csv')

In [ ]:
print(train.shape)

(891, 11)


In [ ]:
train.head

<bound method NDFrame.head of      PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                     

In [ ]:
NAs = pd.concat([train.isnull().sum()], axis=1, keys=["Train"])
NAs.loc[NAs.sum(axis=1) > 0]

,Train
Age,177
Cabin,687
Embarked,2


In [ ]:
train["Age"]=train["Age"].fillna(train["Age"].mean())

In [ ]:
train["Embarked"]=train["Embarked"].fillna(train["Embarked"].mode()[0])

In [ ]:
train["Cabin"]=train["Cabin"].fillna(train["Cabin"].mode()[0])

In [ ]:
train["Pclass"]=train["Pclass"].apply(str)

In [ ]:
for col in train.dtypes[train.dtypes=="object"].index:
    for_dummy=train.pop(col)
    train = pd.concat([train,pd.get_dummies(for_dummy,prefix=col)],axis=1)
train.head()

,PassengerId,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,"Name_Abbing, Mr. Anthony",...,Cabin_F G73,Cabin_F2,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_T,Embarked_C,Embarked_Q,Embarked_S
0,1,0,22.0,1,0,7.2500,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,2,1,38.0,1,0,71.2833,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,1,26.0,0,0,7.9250,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,4,1,35.0,1,0,53.1000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,0,35.0,0,0,8.0500,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
labels=train.pop("Survived")

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(train,labels,test_size=0.25)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
x_train['PassengerId'] = x_train['PassengerId'].astype(float)


In [ ]:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

RandomForestClassifier(max_features='auto', n_estimators=10)

In [ ]:
rf = RandomForestClassifier()
rf.fit(x_train.select_dtypes("number"), y_train)

RandomForestClassifier()

In [ ]:
y_pred = np.array([0, 1, 0, 1])

In [ ]:
from sklearn.metrics import roc_curve, auc

y_test = np.array([1, 0, 1, 0, 1])
y_pred = np.array([0.8, 0.2, 0.9, 0.1, 0.7])

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)

print(roc_auc)

1.0


In [ ]:
n_estimators=[1,2,4,16,32,64,100,200]
train_results = []
test_results= []
for estimator in n_estimators:
  rf = RandomForestClassifier(n_estimators=estimator, n_jobs=-1)

  rf.fit(x_train, y_train)

  train_pred=rf.predict(x_train)
  false_positive_rate, true_positive_rate, threshold = roc_curve(y_train,train_pred)
  roc_auc = auc(false_positive_rate,true_positive_rate)
  train_results.append(roc_auc)
  y_pred=rf.predict(x_test)

  roc_auc=auc(false_positive_rate,true_positive_rate)
  test_results.append(roc_auc)
  false_positive_rate,true_positive_rate, threshold = roc_curve(y_test,y_pred)
  np.array()
  X = X.reshape(X.shape[5:223])
  X = X.transpose()
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(n_estimators, train_results, "b", label="Train AUC")
line2, = plt.plot(n_estimators, test_results, "r", label="Test AUC")
plt.legend(handles=[line1, line2])
plt.ylabel("AUC score")
plt.xlabel("n_estimators")
plt.show()

ValueError: Found input variables with inconsistent numbers of samples: [5, 223]